In [7]:
import pandas as pd
import requests
import json

In [49]:
#dataframe to json format
def df_to_json(df):
    lists = []
    for idx, row in data_csv.iterrows():
        lists.append({"timestamp": row.timestamp, "value": row.value})
    df = {'granularity': 'minutely', 'series': lists}
    return df

In [9]:
def send_request(endpoint, url, subscription_key, request_data):
    headers = {'Content-Type': 'application/json', 'Ocp-Apim-Subscription-Key': subscription_key}
    response = requests.post(endpoint+url, data=json.dumps(request_data), headers=headers)
    return json.loads(response.content.decode("utf-8"))

In [10]:
#Case 1. if you try to make inference on batch size dataset
def detect_batch(request_data):
    print("Detecting anomalies as a batch")
    result = send_request(endpoint, batch_detection_url, subscription_key, request_data)
    print(json.dumps(result, indent=4))

    if result.get('code') != None:
        print("Detection failed. ErrorCode:{}, ErrorMessage:{}".format(result['code'], result['message']))
    else:
        # Find and display the positions of anomalies in the data set
        anomalies = result["isAnomaly"]
        print("Anomalies detected in the following data positions:")
        for x in range(len(anomalies)):
            if anomalies[x] == True:
                print (x)

In [11]:
#Case 2. if you try to make inference on a latest datapoint
def detect_latest(request_data):
    print("Determining if latest data point is an anomaly")
    # send the request, and print the JSON result
    result = send_request(endpoint, latest_point_detection_url, subscription_key, request_data)
    print(json.dumps(result, indent=4))

In [22]:
batch_detection_url = "/anomalydetector/v1.0/timeseries/entire/detect"
latest_point_detection_url = "/anomalydetector/v1.0/timeseries/last/detect"

endpoint = "https://westus2.api.cognitive.microsoft.com/"
subscription_key = "039b14e520f64e5b9907af6ac58edff2"

###if you have json dataset file
data_location = "***Write down your json file path***"
file_handler = open(data_location)
json_data = json.load(file_handler)

###if you have csv datset file: uncomment below comments
###sample csv dataset
# csv_location = "https://raw.githubusercontent.com/numenta/NAB/master/data/realKnownCause/ec2_request_latency_system_failure.csv"
# data_csv = pd.read_csv(csv_location)
# data_csv = data_csv.drop_duplicates(subset = 'timestamp', keep = 'first')
# json_data = df_to_json(data_csv)

#Case 1. if you try to make inference on batch size dataset
detect_batch(json_data)
#Case 2. if you try to make inference on a latest datapoint
detect_latest(json_data)